<a href="https://colab.research.google.com/github/GauravAero/Critical_Thinker_Crewai/blob/main/Critical_Thinker.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Using CrewAI for Defining and Running AI Agents2


In this notebook, we will use **CrewAI**, a framework for defining and running AI agents for specific tasks. This notebook will guide you through setting up an agent, assigning a task, and running the process.

## Steps Overview
1. Define an Agent with a role, goal, and the LLM model it will use.
2. Define a Task that associates a description and expected output to the agent.
3. Initialize a Crew with the agent and task.
4. Kickoff the Crew with an input query and observe the response.

---
### Prerequisites
- Install the `crewai` library.
- Obtain API keys for the LLM model you want to use (e.g., Groq or OpenAI).
- Set up a Python environment with necessary dependencies.

---
### Code Walkthrough
Below is the implementation to define and use an AI agent for answering queries.


In [2]:
## initial setup for Colab.
## if you are using this in local jupyter notebook, skip this cell
!pip install crewai
!pip install langchain_openai
!pip install google-colab

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.3/40.3 kB 2.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.8/42.8 kB 3.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 4.3 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.5/48.5 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 366.7/366.7 kB 14.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.6/8.6 MB 88.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.4/16.4 MB 92.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.5/19.5 MB 55.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.5/119.5 kB 9.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.6/65.6 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.5/72.5 kB 4.5 MB/s eta 0

In [3]:
from crewai import Agent, Task, Crew, Process
import os
from google.colab import userdata

# Retrieve your API key from Colab secrets
os.environ["GOOGLE_API_KEY"] = userdata.get('Gemini_key')

# Method 1: Use CrewAI's built-in LLM configuration (Recommended)
from crewai import LLM

# Configure Gemini LLM directly through CrewAI
gemini_llm = LLM(
    model="gemini/gemini-2.5-flash",  # Use gemini/ prefix for CrewAI
    api_key=os.environ["GOOGLE_API_KEY"]
)

# Define your agent with Gemini LLM
chat_bot = Agent(
    role='Critical Thinker',
    goal='Analyze the text and identify any conflicting information within it',
    llm=gemini_llm,
    verbose=True,
    backstory=(
        'You are a critical thinker who understands details very well and is an expert negotiator. '
        'You can identify conflicting statements and information in given text.'
    ),
    allow_delegation=False,
    max_iter=3
)

# Define a task with a description and expected output
chat_task = Task(
    description='Find if there are any conflicting statements or information in the following text: {text}',
    expected_output="Analyze the text and respond with either 'conflict' if contradictory information is found, or 'no conflict' if the text is consistent. Provide a brief explanation of your reasoning.",
    agent=chat_bot
)

# Define the Crew with agents and tasks
crew = Crew(
    agents=[chat_bot],
    tasks=[chat_task],
    verbose=True,
    process=Process.sequential
)

# Test text with clear conflict
Text = "After a long day at office, I was going back home in the late evening. Then, I met my friend on the way to office."

try:
    # Kickoff the Crew with the input query
    result = crew.kickoff(inputs={'text': Text})
    print("Response:", result)

except Exception as e:
    print(f"Method 1 failed: {e}")
    print("Trying Method 2...")

    # Method 2: Alternative configuration using string model specification
    try:
        # Update agent with string-based model specification
        chat_bot_alt = Agent(
            role='Critical Thinker',
            goal='Analyze the text and identify any conflicting information within it',
            llm="gemini/gemini-2.5-flash",  # Direct string specification
            verbose=True,
            backstory=(
                'You are a critical thinker who understands details very well and is an expert negotiator. '
                'You can identify conflicting statements and information in given text.'
            ),
            allow_delegation=False,
            max_iter=3
        )

        # Update task with new agent
        chat_task_alt = Task(
            description='Find if there are any conflicting statements or information in the following text: {text}',
            expected_output="Analyze the text and respond with either 'conflict' if contradictory information is found, or 'no conflict' if the text is consistent. Provide a brief explanation of your reasoning.",
            agent=chat_bot_alt
        )

        # Create new crew with updated agent
        crew_alt = Crew(
            agents=[chat_bot_alt],
            tasks=[chat_task_alt],
            verbose=True,
            process=Process.sequential
        )

        result = crew_alt.kickoff(inputs={'text': Text})
        print("Response with Method 2:", result)

    except Exception as e2:
        print(f"Method 2 also failed: {e2}")
        print("Trying Method 3...")

        # Method 3: Use environment variable configuration
        try:
            os.environ["GEMINI_MODEL_NAME"] = "gemini-2.5-flash"
            os.environ["GEMINI_API_BASE"] = "https://generativelanguage.googleapis.com/v1beta/"

            # Simple agent configuration
            chat_bot_simple = Agent(
                role='Critical Thinker',
                goal='Analyze the text and identify any conflicting information within it',
                verbose=True,
                backstory=(
                    'You are a critical thinker who understands details very well and is an expert negotiator. '
                    'You can identify conflicting statements and information in given text.'
                ),
                allow_delegation=False,
                max_iter=3
            )

            chat_task_simple = Task(
                description='Find if there are any conflicting statements or information in the following text: {text}',
                expected_output="Analyze the text and respond with either 'conflict' if contradictory information is found, or 'no conflict' if the text is consistent. Provide a brief explanation of your reasoning.",
                agent=chat_bot_simple
            )

            crew_simple = Crew(
                agents=[chat_bot_simple],
                tasks=[chat_task_simple],
                verbose=True,
                process=Process.sequential
            )

            result = crew_simple.kickoff(inputs={'text': Text})
            print("Response with Method 3:", result)

        except Exception as e3:
            print(f"All methods failed. Final error: {e3}")
            print("Please ensure:")
            print("1. Your Gemini API key is correct")
            print("2. You have the latest versions of crewai and langchain-google-genai")
            print("3. Your API key has access to Gemini models")
            print("Try running: !pip install --upgrade crewai langchain-google-genai")

╭──────────────────────────────────────────── Crew Execution Started ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Started                                                                                         │
│  Name: crew                                                                                                     │
│  ID: 44b4babf-5a53-44fb-abe8-d216542b2230                                                                       │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─────────────────────────────────────────────── 🤖 Agent Started ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Critical Thinker                                                                                        │
│                                                                                                                 │
│  Task: Find if there are any conflicting statements or information in the following text: After a long day at   │
│  office, I was going back home in the late evening. Then, I met my friend on the way to office.                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Output()

╭───────────────────────────────────────────── ✅ Agent Final Answer ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Critical Thinker                                                                                        │
│                                                                                                                 │
│  Final Answer:                                                                                                  │
│  conflict - The statement "I was going back home" contradicts "on the way to office." One cannot be             │
│  simultaneously going home from the office and on the way to the office.                                        │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────── Task Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Task Completed                                                                                                 │
│  Name: 82441e06-23d5-4121-8812-e55790987569                                                                     │
│  Agent: Critical Thinker                                                                                        │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────── Crew Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Completed                                                                                       │
│  Name: crew                                                                                                     │
│  ID: 44b4babf-5a53-44fb-abe8-d216542b2230                                                                       │
│  Tool Args:                                                                                                     │
│  Final Output: conflict - The statement "I was going back home" contradicts "on the way to office." One cannot  │
│  be simultaneously going home from the office and on the way to the office.                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Response: conflict - The statement "I was going back home" contradicts "on the way to office." One cannot be simultaneously going home from the office and on the way to the office.
